In [1]:
import torch
import torchtext
import pkuseg
seg=pkuseg.pkuseg()

C:\Users\lazybug\Anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 构建词典  torchtext.vocab.build_vocab_from_iterator
# (运用torchtext.vocab.Vocab)

In [3]:
#生产词list
def yield_cn():   
    with open('cmn.txt',encoding='utf-8') as f:
#         c=1
        for line in f.readlines():
#             print(c)
#             c+=1
            yield seg.cut(line.split('\t')[1])
            
def yield_en():   
    with open('cmn.txt',encoding='utf-8') as f:
#         c=1
        for line in f.readlines():
#             print(c)
#             c+=1
            yield line.split('\t')[0].lower()[:-1].split()+[line.split('\t')[0].lower()[-1]]

In [4]:
x=yield_cn()
y=yield_en()


In [5]:
next(x),next(y)

(['嗨', '。'], ['hi', '.'])

In [6]:
cn_vocab=torchtext.vocab.build_vocab_from_iterator(yield_cn(),specials=['<pad>',"<bos>","<eos>",'<unk>'])
en_vocab=torchtext.vocab.build_vocab_from_iterator(yield_en(),specials=['<pad>',"<bos>","<eos>",'<unk>'])

In [7]:
# cn_vocab.get_itos()[:10]
# en_vocab.get_itos()[:10]

In [8]:
len(en_vocab),len(cn_vocab)

(8308, 14410)

# 切词 ->list[list[str]]

In [9]:
cn_sents=[s for s in yield_cn()]
en_sents=[s for s in yield_en()]

In [10]:
cn_sents[:10]

[['嗨', '。'],
 ['你好', '。'],
 ['你', '用', '跑', '的', '。'],
 ['住手', '！'],
 ['等等', '！'],
 ['等', '一下', '！'],
 ['开始', '！'],
 ['你好', '。'],
 ['我', '试试', '。'],
 ['我', '赢', '了', '。']]

In [11]:
en_sents[:10]

[['hi', '.'],
 ['hi', '.'],
 ['run', '.'],
 ['stop', '!'],
 ['wait', '!'],
 ['wait', '!'],
 ['begin', '.'],
 ['hello', '!'],
 ['i', 'try', '.'],
 ['i', 'won', '!']]

# 看一下最长len

In [12]:
max_len=0
for s in yield_cn():
      if len(s)>max_len:
            max_len=len(s)
max_len

31

# list[list[str]] 转int ->list[list[int]] ,截断,添special symbol, 填充
# 或先截断,添Token,最后再转int(好处在于前面不用换成id)
# 运用torchtext.transforms

In [13]:
max_seq_len=50

In [14]:
en_transform=torchtext.transforms.Sequential(
    
    torchtext.transforms.Truncate(max_seq_len-1),
    torchtext.transforms.AddToken("<eos>",begin=False),
    torchtext.transforms.VocabTransform(en_vocab),
    torchtext.transforms.ToTensor(padding_value=en_vocab['<pad>']),
    torchtext.transforms.PadTransform(max_length=max_seq_len,pad_value=en_vocab["<pad>"]),#PadTransform的input要为tensor
    
    
)

cn_transform_x=torchtext.transforms.Sequential(
    
    torchtext.transforms.Truncate(max_seq_len-2),
    torchtext.transforms.AddToken("<bos>",begin=True),
    torchtext.transforms.AddToken("<eos>",begin=False),
    torchtext.transforms.VocabTransform(cn_vocab),
    torchtext.transforms.ToTensor(padding_value=cn_vocab['<pad>']),
    torchtext.transforms.PadTransform(max_length=max_seq_len,pad_value=cn_vocab["<pad>"]),#PadTransform的input要为tensor
    
)

cn_transform_y=torchtext.transforms.Sequential(
    
    torchtext.transforms.Truncate(max_seq_len-1),
    torchtext.transforms.AddToken("<eos>",begin=False),
    torchtext.transforms.VocabTransform(cn_vocab),
    torchtext.transforms.ToTensor(padding_value=cn_vocab['<pad>']),
    torchtext.transforms.PadTransform(max_length=max_seq_len,pad_value=cn_vocab["<pad>"]),#PadTransform的input要为tensor
    
)

In [15]:
en_data=en_transform(en_sents)
cn_x=cn_transform_x(cn_sents)
cn_y=cn_transform_y(cn_sents)

# 验证

In [16]:
# print("en")
# for sent in en_data[:10]:
#     print(sent[:10])

# print("-"*30)
# print("cn_x")
# for sent in cn_x[:10]:
#     print(sent[:10])

# print("-"*30)
# print("cn_y")
# for sent in cn_y[:10]:
#     print(sent[:10])

# 简单转回token 验证
# 运用torchtext.vocab.Vocab

In [17]:
# print("en")
# for sent in en_data[:10]:
#     print(en_vocab.lookup_tokens(sent[:10].tolist()))

# print("-"*30)
# print("cn_x")
# for sent in cn_x[:10]:
#     print(cn_vocab.lookup_tokens(sent[:10].tolist()))

# print("-"*30)
# print("cn_y")
# for sent in cn_y[:10]:
#     print(cn_vocab.lookup_tokens(sent[:10].tolist()))

# 构建dataset

In [18]:
class CMNDataset(torch.utils.data.Dataset):
    def __init__(self,endata,cndata,cny):
        self.endata=endata
        self.cndata=cndata
        self.cny=cny
    
    def __getitem__(self,index):
        return self.endata[index],self.cndata[index],self.cny[index]
    
    def __len__(self):
        return len(self.endata)

In [19]:
len(en_sents),len(cn_sents)

(29155, 29155)

In [20]:
tot=len(en_sents)
train_size=int(tot*0.6)
eval_size=int(tot*0.2)
test_size=tot-train_size-eval_size

In [21]:
train=CMNDataset(en_data[:train_size],cn_x[:train_size],cn_y[:train_size])
valid=CMNDataset(en_data[train_size:train_size+eval_size],cn_x[train_size:train_size+eval_size],cn_y[train_size:train_size+eval_size])
test=CMNDataset(en_data[train_size+eval_size:],cn_x[train_size+eval_size:],cn_y[train_size+eval_size:])

In [22]:
train_loader=torch.utils.data.DataLoader(train,batch_size=32,shuffle=True,drop_last=True)
valid_loader=torch.utils.data.DataLoader(valid,batch_size=32,shuffle=True,drop_last=True)
test_loader=torch.utils.data.DataLoader(test,batch_size=32,shuffle=True,drop_last=True)

In [28]:
for e in train_loader:
    en,cnx,cny=e
    
#     验证
    print("en")
    for sent in en[:10]:
        print(en_vocab.lookup_tokens(sent[:10].tolist()))

    print("-"*30)
    print("cn_x")
    for sent in cnx[:10]:
        print(cn_vocab.lookup_tokens(sent[:10].tolist()))

    print("-"*30)
    print("cn_y")
    for sent in cny[:10]:
        print(cn_vocab.lookup_tokens(sent[:10].tolist()))

en
['i', 'agreed', 'to', 'the', 'proposal', '.', '<eos>', '<pad>', '<pad>', '<pad>']
['your', 'room', 'is', 'big', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>']
['follow', 'him', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
["i'm", 'not', 'afraid', 'to', 'die', '.', '<eos>', '<pad>', '<pad>', '<pad>']
['he', 'occasionally', 'visited', 'me', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>']
['tom', 'clenched', 'the', 'shovel', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>']
['how', 'am', 'i', 'going', 'to', 'explain', 'this', '?', '<eos>', '<pad>']
['he', 'wants', 'more', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['he', 'left', 'the', 'room', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>']
['but', "you're", 'not', 'there', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>']
------------------------------
cn_x
['<bos>', '我', '同意', '這項', '建議', '。', '<eos>', '<pad>', '<pad>', '<pad>']
['<bos>', '你们', '的', '房间', '很', '大', '。', '<eos>', '<pad>', 

en
["i'll", 'get', 'in', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['do', 'you', 'know', 'whose', 'this', 'is', '?', '<eos>', '<pad>', '<pad>']
['i', 'was', 'tired,', 'so', 'i', "didn't", 'go', '.', '<eos>', '<pad>']
['how', 'long', 'will', 'the', 'meeting', 'last', '?', '<eos>', '<pad>', '<pad>']
['i', 'need', 'to', 'know', 'what', 'you', 'know', '.', '<eos>', '<pad>']
['tom', "doesn't", 'eat', 'red', 'meat', '.', '<eos>', '<pad>', '<pad>', '<pad>']
['please', 'pass', 'me', 'the', 'butter', '.', '<eos>', '<pad>', '<pad>', '<pad>']
['could', 'i', 'park', 'my', 'car', 'here', '?', '<eos>', '<pad>', '<pad>']
['your', 'english', 'is', 'improving', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>']
['why', "don't", 'i', 'call', 'you', '?', '<eos>', '<pad>', '<pad>', '<pad>']
------------------------------
cn_x
['<bos>', '我', '會進', '去', '。', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>']
['<bos>', '你', '知道', '这是', '谁', '的', '吗', '？', '<eos>', '<pad>']
['<bos>', '我', '累', '了', 

en
['where', 'do', 'you', 'work', '?', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>']
['do', 'you', 'think', 'tom', 'is', 'too', 'young', '?', '<eos>', '<pad>']
['i', 'study', 'french', 'a', 'lot', '.', '<eos>', '<pad>', '<pad>', '<pad>']
['he', 'only', 'had', '100', 'dollars', '.', '<eos>', '<pad>', '<pad>', '<pad>']
['we', 'talked', 'over', 'the', 'phone', '.', '<eos>', '<pad>', '<pad>', '<pad>']
['i', 'asked', 'tom', 'to', 'close', 'the', 'door', '.', '<eos>', '<pad>']
["don't", 'cry', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['tom', 'clapped', 'his', 'hands', 'together', '.', '<eos>', '<pad>', '<pad>', '<pad>']
["he's", 'sleeping', 'like', 'a', 'baby', '.', '<eos>', '<pad>', '<pad>', '<pad>']
['how', 'much', 'does', 'he', 'earn', 'per', 'month', '?', '<eos>', '<pad>']
------------------------------
cn_x
['<bos>', '您', '在', '哪里', '工作', '？', '<eos>', '<pad>', '<pad>', '<pad>']
['<bos>', '你', '觉得', '汤姆', '是', '不是', '太', '嫩', '了', '？']
['<bos>', '我', '學', '了', 

# 中文切词 pkuseg

In [24]:
import pkuseg 
seg=pkuseg.pkuseg()

In [25]:
t=seg.cut("用空气来洗手，哈口气判断疾病，精确搜寻雾霾来源 昨天晚上，在浙江大学2016年学术年会开幕式上，一系列脑洞大开的学术成果获得表彰。")

In [26]:
type(t)

list

# GPU使用

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CbowModel(dataset.len_of_vocab(), embedding_dim)
model.to(device)


for epoch in tqdm(range(num_epoch), desc="Training"):
    inputs, targets = [x.to(device) for x in dataset.getdata()]